In [2]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from scipy.stats import pointbiserialr
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [3]:
#connection
conn = snowflake.connector.Connect(user='ANALYST', password='Fedewap2023!', account='rl43903.east-us-2.azure')

In [4]:
#sql query
sql = "SELECT main.*, sub.COUNT_SINGLE_ADVANCE, sub.COUNT_COMPLIMENTARY, taskdata.TASK_COUNT_EMAIL, taskdata.TASK_COUNT_LISTEMAIL, taskdata.TASK_COUNT_CALL, taskdata.TASK_COUNT_TASK, taskdata.TASK_COUNT_CADENCE FROM (SELECT model.DW_ID, model.FIRST_NAME_C, model.LAST_NAME_C, model.seasonyear, model.RENEWED_NEXT_YEAR, model.MLS_TENURE, model.ORDER_QUANTITY_M, model.ORDER_QUANTITY_M_LAST_YEAR, model.ORDER_QUANTITY_M_ALL_TIME, model.ORDER_SPEND_M, model.ORDER_SPEND_M_LAST_YEAR, model.ORDER_SPEND_M_ALL_TIME, model.TICKET_QUANTITY_T, model.TICKET_QUANTITY_LAST_YEAR, model.TICKET_QUANTITY_ALL_TIME, model.ATTENDED_EVENTS, model.ATTENDED_EVENTS_LAST_YEAR, model.ATTENDED_EVENTS_ALL_TIME, model.TICKET_SPEND_T, model.TICKET_SPEND_T_LAST_YEAR, model.TICKET_SPEND_T_ALL_TIME, model.SINGLE_GAME_FLAG, model.PREV_SINGLE_GAME_FLAG, model.COMP_FLAG, model.PREV_COMP_FLAG, model.GROUP_FLAG, model.PREV_GROUP_FLAG, model.FULL_SEASON_FLAG, model.PREV_FULL_SEASON_FLAG, model.PARTIAL_SEASON_FLAG, model.PREV_PARTIAL_SEASON_FLAG, model.prev_yr_events_attended, model.DISTANCE_TO_STADIUM_C FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model WHERE model.SEASON_TICKET_HOLDER = 1 AND model.seasonyear LIKE ANY (2021, 2022, 2023)) as main LEFT JOIN (SELECT id_table.DW_ID, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = '6-Single Advance' THEN 1 ELSE 0 END) AS COUNT_SINGLE_ADVANCE, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = 'Complimentary' THEN 1 ELSE 0 END) AS COUNT_COMPLIMENTARY FROM (SELECT model.DW_ID, master.ACCT_ID_T FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model LEFT JOIN CHICAGOFIRE_DATABASE.MLS.MASTER_CF as master ON model.DW_ID = master.DW_ID_MASTER WHERE model.SEASONYEAR in (2021, 2022) and model.SEASON_TICKET_HOLDER = 1) AS id_table INNER JOIN CHICAGOFIRE_DATABASE.ARCHTICS.ARCHITCS_2022 as arch ON id_table.ACCT_ID_T = arch.ACCT_ID GROUP BY id_table.DW_ID) as sub ON main.DW_ID = sub.DW_ID LEFT JOIN (SELECT model.DW_ID, SUM(CASE WHEN task.TASKSUBTYPE = 'Email' THEN 1 ELSE 0 END) AS TASK_COUNT_EMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'List Email' THEN 1 ELSE 0 END) AS TASK_COUNT_LISTEMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'Call' THEN 1 ELSE 0 END) AS TASK_COUNT_CALL, SUM(CASE WHEN task.TASKSUBTYPE = 'Task' THEN 1 ELSE 0 END) AS TASK_COUNT_TASK, SUM(CASE WHEN task.TASKSUBTYPE = 'Cadence' THEN 1 ELSE 0 END) AS TASK_COUNT_CADENCE FROM CHICAGOFIRE_DATABASE.SALESFORCE.SF_OPPORTUNITIES as opp INNER JOIN CHICAGOFIRE_DATABASE.SALESFORCE.SF_TASK as task ON task.WHOID = opp.CONTACTID__C INNER JOIN CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model ON model.ACCT_ID_T = opp.TICKET_ID__C WHERE task.ACTIVITYDATE >= '2023-08-01' and opp.stagename not like 'Closed Lost' GROUP BY model.DW_ID) as taskdata ON main.DW_ID = taskdata.DW_ID"
#sql23 = "SELECT main.*, sub.COUNT_SINGLE_ADVANCE, sub.COUNT_COMPLIMENTARY, taskdata.TASK_COUNT_EMAIL, taskdata.TASK_COUNT_LISTEMAIL, taskdata.TASK_COUNT_CALL, taskdata.TASK_COUNT_TASK, taskdata.TASK_COUNT_CADENCE FROM (SELECT model.DW_ID, model.FIRST_NAME_C, model.LAST_NAME_C, model.seasonyear, model.RENEWED_NEXT_YEAR, model.MLS_TENURE, model.ORDER_QUANTITY_M, model.ORDER_QUANTITY_M_LAST_YEAR, model.ORDER_QUANTITY_M_ALL_TIME, model.ORDER_SPEND_M, model.ORDER_SPEND_M_LAST_YEAR, model.ORDER_SPEND_M_ALL_TIME, model.TICKET_QUANTITY_T, model.TICKET_QUANTITY_LAST_YEAR, model.TICKET_QUANTITY_ALL_TIME, model.COUNT_OF_MLS_EVENTS, model.COUNT_OF_MLS_EVENTS_LAST_YEAR, model.COUNT_OF_MLS_EVENTS_ALL_TIME, model.TICKET_SPEND_T, model.TICKET_SPEND_T_LAST_YEAR, model.TICKET_SPEND_T_ALL_TIME, model.SINGLE_GAME_FLAG, model.PREV_SINGLE_GAME_FLAG, model.COMP_FLAG, model.PREV_COMP_FLAG, model.GROUP_FLAG, model.PREV_GROUP_FLAG, model.FULL_SEASON_FLAG, model.PREV_FULL_SEASON_FLAG, model.PARTIAL_SEASON_FLAG, model.PREV_PARTIAL_SEASON_FLAG, model.annual_events_attended, model.prev_yr_events_attended, model.DISTANCE_TO_STADIUM_C FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model WHERE model.SEASON_TICKET_HOLDER = 1 AND model.seasonyear LIKE ANY (2023)) as main LEFT JOIN (SELECT id_table.DW_ID, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = '6-Single Advance' THEN 1 ELSE 0 END) AS COUNT_SINGLE_ADVANCE, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = 'Complimentary' THEN 1 ELSE 0 END) AS COUNT_COMPLIMENTARY FROM (SELECT model.DW_ID, master.ACCT_ID_T FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model LEFT JOIN CHICAGOFIRE_DATABASE.MLS.MASTER_CF as master ON model.DW_ID = master.DW_ID_MASTER WHERE model.SEASONYEAR in (2021, 2022, 2023) and model.SEASON_TICKET_HOLDER = 1) AS id_table INNER JOIN CHICAGOFIRE_DATABASE.ARCHTICS.ARCHITCS_2022 as arch ON id_table.ACCT_ID_T = arch.ACCT_ID GROUP BY id_table.DW_ID) as sub ON main.DW_ID = sub.DW_ID LEFT JOIN (SELECT model.DW_ID, SUM(CASE WHEN task.TASKSUBTYPE = 'Email' THEN 1 ELSE 0 END) AS TASK_COUNT_EMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'List Email' THEN 1 ELSE 0 END) AS TASK_COUNT_LISTEMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'Call' THEN 1 ELSE 0 END) AS TASK_COUNT_CALL, SUM(CASE WHEN task.TASKSUBTYPE = 'Task' THEN 1 ELSE 0 END) AS TASK_COUNT_TASK, SUM(CASE WHEN task.TASKSUBTYPE = 'Cadence' THEN 1 ELSE 0 END) AS TASK_COUNT_CADENCE FROM CHICAGOFIRE_DATABASE.SALESFORCE.SF_OPPORTUNITIES as opp INNER JOIN CHICAGOFIRE_DATABASE.SALESFORCE.SF_TASK as task ON task.WHOID = opp.CONTACTID__C INNER JOIN CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model ON model.ACCT_ID_T = opp.TICKET_ID__C WHERE task.ACTIVITYDATE >= '2023-06-01' GROUP BY model.DW_ID) as taskdata ON main.DW_ID = taskdata.DW_ID"

In [5]:
#load in dataset
res = conn.cursor().execute(sql).fetchall()
df = pd.read_sql(sql, conn)
df23 = df[df.SEASONYEAR == '2023']
df = df[(df.SEASONYEAR == '2022') | (df.SEASONYEAR == '2021')]
df23.head()

C:\Users\dgilbert\AppData\Local\Temp\ipykernel_16264\3406225740.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,PREV_PARTIAL_SEASON_FLAG,PREV_YR_EVENTS_ATTENDED,DISTANCE_TO_STADIUM_C,COUNT_SINGLE_ADVANCE,COUNT_COMPLIMENTARY,TASK_COUNT_EMAIL,TASK_COUNT_LISTEMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK,TASK_COUNT_CADENCE
0,18213239,David,Miller,2023,1,215,0,0,0,0.0,...,0,0,7.0,NaN,NaN,0.0,0.0,30.0,30.0,0.0
1,18240005,Carlos,Contreras,2023,0,322,0,0,0,0.0,...,0,0,4.0,NaN,NaN,0.0,0.0,30.0,30.0,0.0
2,18281907,Stephen,Moy,2023,1,205,0,0,0,0.0,...,0,0,5.0,NaN,NaN,0.0,0.0,20.0,0.0,0.0
3,18313257,Nicholas,Byrne,2023,1,201,0,0,0,0.0,...,0,0,15.0,NaN,NaN,80.0,0.0,400.0,120.0,0.0
4,18398592,Mark,Bradley,2023,0,187,0,0,0,0.0,...,0,0,187.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning

#### Drop vendor IDs

In [6]:
ids_to_drop = [3790591, 295808, 348953, 31405, 114785, 4513005, 3496760]
for id_ in ids_to_drop:
    if id_ in df23.index:
        print(f"ID {id_} is in the DataFrame's index.")
    else:
        print(f"ID {id_} is NOT in the DataFrame's index.")


ID 3790591 is NOT in the DataFrame's index.
ID 295808 is NOT in the DataFrame's index.
ID 348953 is NOT in the DataFrame's index.
ID 31405 is NOT in the DataFrame's index.
ID 114785 is NOT in the DataFrame's index.
ID 4513005 is NOT in the DataFrame's index.
ID 3496760 is NOT in the DataFrame's index.


In [7]:
ids_to_drop = [3790591, 31405, 4513005]
df.drop(ids_to_drop, inplace=True)
df23.drop(ids_to_drop, inplace=True)

KeyError: '[3790591, 31405, 4513005] not found in axis'

#### Fill nulls

In [8]:
df['COUNT_SINGLE_ADVANCE'] = df['COUNT_SINGLE_ADVANCE'].fillna(0).astype(int)
df['COUNT_COMPLIMENTARY'] = df['COUNT_COMPLIMENTARY'].fillna(0).astype(int)
df['TASK_COUNT_EMAIL'] = df['TASK_COUNT_EMAIL'].fillna(0).astype(int)
df['TASK_COUNT_LISTEMAIL'] = df['TASK_COUNT_LISTEMAIL'].fillna(0).astype(int)
df['TASK_COUNT_CALL'] = df['TASK_COUNT_CALL'].fillna(0).astype(int)
df['TASK_COUNT_TASK'] = df['TASK_COUNT_TASK'].fillna(0).astype(int)
df['TASK_COUNT_CADENCE'] = df['TASK_COUNT_CADENCE'].fillna(0).astype(int)

df23['COUNT_SINGLE_ADVANCE'] = df23['COUNT_SINGLE_ADVANCE'].fillna(0).astype(int)
df23['COUNT_COMPLIMENTARY'] = df23['COUNT_COMPLIMENTARY'].fillna(0).astype(int)
df23['TASK_COUNT_EMAIL'] = df23['TASK_COUNT_EMAIL'].fillna(0).astype(int)
df23['TASK_COUNT_LISTEMAIL'] = df23['TASK_COUNT_LISTEMAIL'].fillna(0).astype(int)
df23['TASK_COUNT_CALL'] = df23['TASK_COUNT_CALL'].fillna(0).astype(int)
df23['TASK_COUNT_TASK'] = df23['TASK_COUNT_TASK'].fillna(0).astype(int)
df23['TASK_COUNT_CADENCE'] = df23['TASK_COUNT_CADENCE'].fillna(0).astype(int)

In [9]:
null_columns = df.columns[df.isnull().any()]
print(null_columns)

null_columns23 = df23.columns[df23.isnull().any()]
print(null_columns23)

Index(['FIRST_NAME_C', 'LAST_NAME_C', 'DISTANCE_TO_STADIUM_C'], dtype='object')
Index(['DISTANCE_TO_STADIUM_C'], dtype='object')


In [10]:
df_features = df
df23_features = df23

## Feature Engineering

In [11]:
df.columns.to_list()
df23.columns

Index(['DW_ID', 'FIRST_NAME_C', 'LAST_NAME_C', 'SEASONYEAR',
       'RENEWED_NEXT_YEAR', 'MLS_TENURE', 'ORDER_QUANTITY_M',
       'ORDER_QUANTITY_M_LAST_YEAR', 'ORDER_QUANTITY_M_ALL_TIME',
       'ORDER_SPEND_M', 'ORDER_SPEND_M_LAST_YEAR', 'ORDER_SPEND_M_ALL_TIME',
       'TICKET_QUANTITY_T', 'TICKET_QUANTITY_LAST_YEAR',
       'TICKET_QUANTITY_ALL_TIME', 'ATTENDED_EVENTS',
       'ATTENDED_EVENTS_LAST_YEAR', 'ATTENDED_EVENTS_ALL_TIME',
       'TICKET_SPEND_T', 'TICKET_SPEND_T_LAST_YEAR', 'TICKET_SPEND_T_ALL_TIME',
       'SINGLE_GAME_FLAG', 'PREV_SINGLE_GAME_FLAG', 'COMP_FLAG',
       'PREV_COMP_FLAG', 'GROUP_FLAG', 'PREV_GROUP_FLAG', 'FULL_SEASON_FLAG',
       'PREV_FULL_SEASON_FLAG', 'PARTIAL_SEASON_FLAG',
       'PREV_PARTIAL_SEASON_FLAG', 'PREV_YR_EVENTS_ATTENDED',
       'DISTANCE_TO_STADIUM_C', 'COUNT_SINGLE_ADVANCE', 'COUNT_COMPLIMENTARY',
       'TASK_COUNT_EMAIL', 'TASK_COUNT_LISTEMAIL', 'TASK_COUNT_CALL',
       'TASK_COUNT_TASK', 'TASK_COUNT_CADENCE'],
      dtype='object')

Tenure

In [12]:
# Convert Tenure to years
df['MLS_TENURE_YEARS'] = df['MLS_TENURE']/365.242374
df[['MLS_TENURE', 'MLS_TENURE_YEARS']]

df23['MLS_TENURE_YEARS'] = df23['MLS_TENURE']/365.242374
df23[['MLS_TENURE', 'MLS_TENURE_YEARS']]


,MLS_TENURE,MLS_TENURE_YEARS
0,215,0.588650
1,322,0.881606
2,205,0.561271
3,201,0.550319
4,187,0.511989
...,...,...
5508,3434,9.401976
5511,4193,11.480048
5514,8787,24.057997
5516,7191,19.688296


Attendance Rate

In [13]:
#Convert attendance to attendance rate
games_21 = 11 #could go to 16
games_22 = 18
games_23 = 22
df['ATTENDANCE_RATE'] = np.nan
# Convert the SEASONYEAR column to integer
df['SEASONYEAR'] = df['SEASONYEAR'].astype('int64')
df.loc[df.SEASONYEAR == 2021, 'ATTENDANCE_RATE'] = df.loc[df.SEASONYEAR == 2021, 'ATTENDED_EVENTS']/games_21
df.loc[df.SEASONYEAR == 2022, 'ATTENDANCE_RATE'] = df.loc[df.SEASONYEAR == 2022, 'ATTENDED_EVENTS']/games_22 
df[['SEASONYEAR', 'ATTENDED_EVENTS', 'ATTENDANCE_RATE']]


df23['ATTENDANCE_RATE'] = np.nan
# Convert the SEASONYEAR column to integer
df23['SEASONYEAR'] = df23['SEASONYEAR'].astype('int64')
df23.loc[df23.SEASONYEAR == 2023, 'ATTENDANCE_RATE'] = df23.loc[df23.SEASONYEAR == 2023, 'ATTENDED_EVENTS']/games_23
df23[['SEASONYEAR', 'ATTENDED_EVENTS', 'ATTENDANCE_RATE']]


,SEASONYEAR,ATTENDED_EVENTS,ATTENDANCE_RATE
0,2023,1,0.045455
1,2023,8,0.363636
2,2023,8,0.363636
3,2023,3,0.136364
4,2023,0,0.000000
...,...,...,...
5508,2023,13,0.590909
5511,2023,5,0.227273
5514,2023,16,0.727273
5516,2023,0,0.000000


#### Pre-Processing

In [14]:
def categorize_spend(spend):
    if spend == 0:
        return 'No Purchase'
    elif spend <= 100:  # example threshold
        return 'Low'
    elif spend <= 300:  # example threshold
        return 'Medium'
    else:
        return 'High'

df['AVG_ORDER_SPEND'] = df['ORDER_SPEND_M']/df['ORDER_QUANTITY_M']
df['AVG_ORDER_SPEND'] = df['AVG_ORDER_SPEND'].fillna(0)
df['AVG_ORDER_SPEND_CAT'] = df['AVG_ORDER_SPEND'].apply(categorize_spend)
df = pd.get_dummies(df, columns=['AVG_ORDER_SPEND_CAT'])
df['AVG_ORDER_SPEND_CAT_High'] = df['AVG_ORDER_SPEND_CAT_High'].astype(int)
df['AVG_ORDER_SPEND_CAT_Medium'] = df['AVG_ORDER_SPEND_CAT_Medium'].astype(int)
df['AVG_ORDER_SPEND_CAT_Low'] = df['AVG_ORDER_SPEND_CAT_Low'].astype(int)
df['AVG_ORDER_SPEND_CAT_No Purchase'] = df['AVG_ORDER_SPEND_CAT_No Purchase'].astype(int)

In [15]:
df23['AVG_ORDER_SPEND'] = df23['ORDER_SPEND_M']/df23['ORDER_QUANTITY_M']
df23['AVG_ORDER_SPEND'] = df23['AVG_ORDER_SPEND'].fillna(0)
df23['AVG_ORDER_SPEND_CAT'] = df23['AVG_ORDER_SPEND'].apply(categorize_spend)
df23 = pd.get_dummies(df23, columns=['AVG_ORDER_SPEND_CAT'])
df23['AVG_ORDER_SPEND_CAT_High'] = df23['AVG_ORDER_SPEND_CAT_High'].astype(int)
df23['AVG_ORDER_SPEND_CAT_Medium'] = df23['AVG_ORDER_SPEND_CAT_Medium'].astype(int)
df23['AVG_ORDER_SPEND_CAT_Low'] = df23['AVG_ORDER_SPEND_CAT_Low'].astype(int)
df23['AVG_ORDER_SPEND_CAT_No Purchase'] = df23['AVG_ORDER_SPEND_CAT_No Purchase'].astype(int)


In [93]:
from sklearn import preprocessing 


df['TENURE_LOG'] = np.log(df['MLS_TENURE_YEARS']).replace(-np.inf, 0)
df['TENURE_NORMAL'] = preprocessing.normalize(df['MLS_TENURE_YEARS'].array.reshape(-1, 1))
df.loc[df.SEASONYEAR == 2021, 'TICKET_SPEND_PER_GAME'] = df.loc[df.SEASONYEAR == 2021, 'TICKET_SPEND_T']/games_21
df.loc[df.SEASONYEAR == 2022, 'TICKET_SPEND_PER_GAME'] = df.loc[df.SEASONYEAR == 2022, 'TICKET_SPEND_T']/games_22
df['TICKET_SPEND_PER_GAME_LOG'] = np.log(df['TICKET_SPEND_PER_GAME']).replace(-np.inf, 0)
df['COUNT_SINGLE_ADVANCE_LOG'] = np.log(df['COUNT_SINGLE_ADVANCE']).replace(-np.inf, 0)
df['COUNT_COMPLIMENTARY_LOG'] = np.log(df['COUNT_COMPLIMENTARY']).replace(-np.inf, 0)
df['TASK_COUNT_EMAIL_LOG'] = np.log(df['TASK_COUNT_EMAIL']).replace(-np.inf, 0)
df['TASK_COUNT_CALL_LOG'] = np.log(df['TASK_COUNT_CALL']).replace(-np.inf, 0)
df['TASK_COUNT_TASK_LOG'] = np.log(df['TASK_COUNT_TASK']).replace(-np.inf, 0)
df['TASK_COUNT_CADENCE_LOG'] = np.log(df['TASK_COUNT_CADENCE']).replace(-np.inf, 0)

df23['TENURE_LOG'] = np.log(df23['MLS_TENURE_YEARS']).replace(-np.inf, 0)
df23['TICKET_SPEND_PER_GAME'] = df23['TICKET_SPEND_T']/games_23
df23['TICKET_SPEND_PER_GAME_LOG'] = np.log(df23['TICKET_SPEND_PER_GAME']).replace(-np.inf, 0)
df23['COUNT_SINGLE_ADVANCE_LOG'] = np.log(df23['COUNT_SINGLE_ADVANCE']).replace(-np.inf, 0)
df23['COUNT_COMPLIMENTARY_LOG'] = np.log(df23['COUNT_COMPLIMENTARY']).replace(-np.inf, 0)
df23['TASK_COUNT_EMAIL_LOG'] = np.log(df23['TASK_COUNT_EMAIL']).replace(-np.inf, 0)
df23['TASK_COUNT_CALL_LOG'] = np.log(df23['TASK_COUNT_CALL']).replace(-np.inf, 0)
df23['TASK_COUNT_TASK_LOG'] = np.log(df23['TASK_COUNT_TASK']).replace(-np.inf, 0)
df23['TASK_COUNT_CADENCE_LOG'] = np.log(df23['TASK_COUNT_CADENCE']).replace(-np.inf, 0)


c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [99]:
df['TENURE_NORMAL'] = (df['MLS_TENURE_YEARS'] - df['MLS_TENURE_YEARS'].mean()) / df['MLS_TENURE_YEARS'].std()

df[['TENURE_LOG', 'TENURE_NORMAL']]

,TENURE_LOG,TENURE_NORMAL
25,1.109751,-0.952025
27,1.677584,-0.497095
29,1.920681,-0.208132
30,1.762377,-0.404176
31,2.295324,0.400411
...,...,...
5519,2.593773,1.077702
5520,2.516148,0.881658
5521,1.658998,-0.516431
5523,2.389480,0.592695


In [17]:
df[['ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME_LOG', 
                 'TENURE_LOG', 
                 'COUNT_SINGLE_ADVANCE_LOG', 'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL_LOG', 
                 'TASK_COUNT_CALL_LOG', 'TASK_COUNT_TASK_LOG'
                 , 'TASK_COUNT_CADENCE_LOG'
                 ]]

,ATTENDANCE_RATE,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_No Purchase,TICKET_SPEND_PER_GAME_LOG,TENURE_LOG,COUNT_SINGLE_ADVANCE_LOG,COUNT_COMPLIMENTARY_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG,TASK_COUNT_CADENCE_LOG
25,0.166667,0,0,0,1,4.873924,1.109751,0.000000,0.000000,3.401197,3.401197,3.401197,0.0
27,0.388889,0,0,0,1,3.866561,1.677584,0.693147,0.000000,3.401197,4.094345,0.000000,0.0
29,0.333333,0,0,0,1,3.101093,1.920681,0.000000,0.000000,0.000000,4.382027,2.995732,0.0
30,0.090909,0,0,0,1,3.618262,1.762377,0.000000,0.000000,0.000000,4.382027,2.995732,0.0
31,0.000000,0,0,0,1,4.462768,2.295324,0.000000,0.693147,0.000000,5.010635,2.302585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5519,0.166667,0,0,0,1,3.878121,2.593773,0.693147,0.000000,0.000000,0.000000,0.000000,0.0
5520,0.090909,0,0,0,1,4.347341,2.516148,0.693147,0.000000,0.000000,0.000000,0.000000,0.0
5521,0.000000,0,0,0,1,1.364467,1.658998,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5523,0.666667,0,0,1,0,3.472656,2.389480,0.693147,0.000000,3.401197,4.094345,0.000000,0.0


In [38]:
df.columns.to_list()

['DW_ID',
 'FIRST_NAME_C',
 'LAST_NAME_C',
 'SEASONYEAR',
 'RENEWED_NEXT_YEAR',
 'MLS_TENURE',
 'ORDER_QUANTITY_M',
 'ORDER_QUANTITY_M_LAST_YEAR',
 'ORDER_QUANTITY_M_ALL_TIME',
 'ORDER_SPEND_M',
 'ORDER_SPEND_M_LAST_YEAR',
 'ORDER_SPEND_M_ALL_TIME',
 'TICKET_QUANTITY_T',
 'TICKET_QUANTITY_LAST_YEAR',
 'TICKET_QUANTITY_ALL_TIME',
 'ATTENDED_EVENTS',
 'ATTENDED_EVENTS_LAST_YEAR',
 'ATTENDED_EVENTS_ALL_TIME',
 'TICKET_SPEND_T',
 'TICKET_SPEND_T_LAST_YEAR',
 'TICKET_SPEND_T_ALL_TIME',
 'SINGLE_GAME_FLAG',
 'PREV_SINGLE_GAME_FLAG',
 'COMP_FLAG',
 'PREV_COMP_FLAG',
 'GROUP_FLAG',
 'PREV_GROUP_FLAG',
 'FULL_SEASON_FLAG',
 'PREV_FULL_SEASON_FLAG',
 'PARTIAL_SEASON_FLAG',
 'PREV_PARTIAL_SEASON_FLAG',
 'PREV_YR_EVENTS_ATTENDED',
 'DISTANCE_TO_STADIUM_C',
 'COUNT_SINGLE_ADVANCE',
 'COUNT_COMPLIMENTARY',
 'TASK_COUNT_EMAIL',
 'TASK_COUNT_LISTEMAIL',
 'TASK_COUNT_CALL',
 'TASK_COUNT_TASK',
 'TASK_COUNT_CADENCE',
 'MLS_TENURE_YEARS',
 'ATTENDANCE_RATE',
 'AVG_ORDER_SPEND',
 'AVG_ORDER_SPEND_CAT_High

In [39]:
random_state = 1010

X = df[['ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME', 
                 'MLS_TENURE', 
                 'COUNT_SINGLE_ADVANCE', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL', 
                 'TASK_COUNT_CALL', 'TASK_COUNT_TASK'
                #, 'TASK_COUNT_CADENCE_LOG'
                 ]]
y = df['RENEWED_NEXT_YEAR']

# Model
model = LogisticRegression(max_iter=1000)

# 11 stratified folds
kf = StratifiedKFold(n_splits=11, random_state=random_state, shuffle=True)

# Lists to store results
accuracy_results = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    accuracy_results.append(accuracy)

# Mean accuracy
mean_accuracy = np.mean(accuracy_results)
print(f"Mean Accuracy: {mean_accuracy}")

Mean Accuracy: 0.752697179481615


In [40]:
result_table = pd.DataFrame([model.feature_names_in_, model.coef_[0]])

In [51]:
X_test

,ATTENDANCE_RATE,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_No Purchase,TICKET_SPEND_PER_GAME,MLS_TENURE,COUNT_SINGLE_ADVANCE,TASK_COUNT_EMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK
56,0.277778,0,0,0,1,88.888889,4515,0,30,120,0
77,0.388889,0,0,0,1,47.222222,2034,0,90,90,30
83,0.388889,0,0,0,1,40.833333,3522,4,0,0,0
97,0.545455,0,0,0,1,10.454545,1616,1,0,0,0
115,0.272727,0,0,0,1,51.818182,695,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5496,0.166667,0,0,0,1,52.777778,6065,0,0,60,30
5497,0.181818,0,0,0,1,62.272727,5700,0,0,60,30
5499,0.000000,0,0,0,1,75.555556,4438,0,0,0,0
5502,0.272727,0,0,0,1,100.909091,2904,0,40,240,40


,0,1,2,3,4,5,6,7,8,9,10
0,ATTENDANCE_RATE,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_No Purchase,TICKET_SPEND_PER_GAME,MLS_TENURE,COUNT_SINGLE_ADVANCE,TASK_COUNT_EMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK
1,2.451024,0.086295,0.033221,-0.360351,-0.602243,0.00143,0.000175,0.055587,0.003052,0.004893,0.012628


In [42]:
X_predict = df23[['ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME', 
                 'MLS_TENURE', 
                 'COUNT_SINGLE_ADVANCE', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL', 
                 'TASK_COUNT_CALL', 'TASK_COUNT_TASK'
                #, 'TASK_COUNT_CADENCE_LOG'
                 ]]

predictions_2023 = model.predict(X_predict)


In [58]:
df23

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,TICKET_SPEND_PER_GAME,TICKET_SPEND_PER_GAME_LOG,COUNT_SINGLE_ADVANCE_LOG,COUNT_COMPLIMENTARY_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG,TASK_COUNT_CADENCE_LOG,PREDICTIONS,renewal_predict_prob
0,18213239,David,Miller,2023,1,215,0,0,0,0.00,...,38.000000,3.637586,0.000000,0.0,0.000000,3.401197,3.401197,0.0,0,0.323682
1,18240005,Carlos,Contreras,2023,0,322,0,0,0,0.00,...,29.090909,3.370426,0.000000,0.0,0.000000,3.401197,3.401197,0.0,1,0.512240
2,18281907,Stephen,Moy,2023,1,205,0,0,0,0.00,...,10.227273,2.325058,0.000000,0.0,0.000000,2.995732,0.000000,0.0,0,0.395018
3,18313257,Nicholas,Byrne,2023,1,201,0,0,0,0.00,...,17.318182,2.851757,0.000000,0.0,4.382027,5.991465,4.787492,0.0,1,0.933695
4,18398592,Mark,Bradley,2023,0,187,0,0,0,0.00,...,5.454545,1.696449,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0,0.193820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5508,11975597,Jorge,Rinconeno,2023,1,3434,0,0,0,0.00,...,48.636364,3.884371,0.000000,0.0,0.000000,5.010635,0.000000,0.0,1,0.800157
5511,11983362,Jim,Castellano,2023,1,4193,0,0,4,0.00,...,57.681818,4.054942,0.000000,0.0,0.000000,5.598422,3.401197,0.0,1,0.833110
5514,11991599,Dmytro,Kurywczak,2023,1,8787,0,0,0,0.00,...,75.738636,4.327288,2.397895,0.0,3.401197,4.094345,0.000000,0.0,1,0.950756
5516,11999535,Larry,Garetto,2023,1,7191,0,0,0,0.00,...,186.000000,5.225747,0.000000,0.0,3.401197,5.347108,3.401197,0.0,1,0.825968


In [44]:
df23['PREDICTIONS'] = predictions_2023

In [45]:
df23['PREDICTIONS']

0       0
1       1
2       0
3       1
4       0
       ..
5508    1
5511    1
5514    1
5516    1
5522    1
Name: PREDICTIONS, Length: 1627, dtype: int64

In [46]:
probabilities_2023 = model.predict_proba(X_predict)

In [64]:
df23['renewal_predict_prob'] = probabilities_2023[:, 1]

In [73]:
#df23.loc[df23['DW_ID'] == 18281907]

df23.loc[df23['DW_ID'] == 18281907][[ 'renewal_predict_prob', 'ATTENDANCE_RATE', 
                 'TICKET_SPEND_PER_GAME', 
                 'MLS_TENURE', 
                 'COUNT_SINGLE_ADVANCE', 'TASK_COUNT_CALL', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL', 
                  'TASK_COUNT_TASK','AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', ]]

,renewal_predict_prob,ATTENDANCE_RATE,TICKET_SPEND_PER_GAME,MLS_TENURE,COUNT_SINGLE_ADVANCE,TASK_COUNT_CALL,TASK_COUNT_EMAIL,TASK_COUNT_TASK,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_No Purchase
2,0.395018,0.363636,10.227273,205,0,20,0,0,0,0,0,1


In [74]:
result_table

,0,1,2,3,4,5,6,7,8,9,10
0,ATTENDANCE_RATE,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_No Purchase,TICKET_SPEND_PER_GAME,MLS_TENURE,COUNT_SINGLE_ADVANCE,TASK_COUNT_EMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK
1,2.451024,0.086295,0.033221,-0.360351,-0.602243,0.00143,0.000175,0.055587,0.003052,0.004893,0.012628


In [81]:
2.45 * 0.363636 + 0.00143*10.227372 + 0.000175 * 205 + 0.004893*20 + 1*-0.6022

0.4370683419600002

In [ ]:
df23.loc[df23.DW_ID == 11983362]
#plt.hist(df23['renewal_predict_prob'], bins=5, edgecolor='black', alpha=0.7, density=True)

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,TICKET_SPEND_PER_GAME,TICKET_SPEND_PER_GAME_LOG,COUNT_SINGLE_ADVANCE_LOG,COUNT_COMPLIMENTARY_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG,TASK_COUNT_CADENCE_LOG,PREDICTIONS,renewal_predict_prob
4579,11983362,Jim,Castellano,2023,1,4103,0,0,4,0.0,...,57.681818,4.054942,0.0,0.0,4.094345,6.697034,4.094345,0.0,1,0.657891


In [ ]:
df23[['DW_ID', 'FIRST_NAME_C', 'LAST_NAME_C', 'renewal_predict_prob', 'ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME_LOG', 
                 'TENURE_LOG',  
                 'COUNT_SINGLE_ADVANCE_LOG', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL_LOG', 
                 'TASK_COUNT_CALL_LOG', 'TASK_COUNT_TASK_LOG'
                #, 'TASK_COUNT_CADENCE_LOG'
                ]].to_csv("Output.csv")

In [ ]:
df_ids = pd.read_csv("no_ids_preds.csv")

In [ ]:
sql2 = "SELECT main.*, sub.COUNT_SINGLE_ADVANCE, sub.COUNT_COMPLIMENTARY, taskdata.TASK_COUNT_EMAIL, taskdata.TASK_COUNT_LISTEMAIL, taskdata.TASK_COUNT_CALL, taskdata.TASK_COUNT_TASK, taskdata.TASK_COUNT_CADENCE FROM (SELECT model.DW_ID, model.FIRST_NAME_C, model.LAST_NAME_C, model.seasonyear, model.RENEWED_NEXT_YEAR, model.MLS_TENURE, model.ORDER_QUANTITY_M, model.ORDER_QUANTITY_M_LAST_YEAR, model.ORDER_QUANTITY_M_ALL_TIME, model.ORDER_SPEND_M, model.ORDER_SPEND_M_LAST_YEAR, model.ORDER_SPEND_M_ALL_TIME, model.TICKET_QUANTITY_T, model.TICKET_QUANTITY_LAST_YEAR, model.TICKET_QUANTITY_ALL_TIME, model.ATTENDED_EVENTS, model.ATTENDED_EVENTS_LAST_YEAR, model.ATTENDED_EVENTS_ALL_TIME, model.TICKET_SPEND_T, model.TICKET_SPEND_T_LAST_YEAR, model.TICKET_SPEND_T_ALL_TIME, model.SINGLE_GAME_FLAG, model.PREV_SINGLE_GAME_FLAG, model.COMP_FLAG, model.PREV_COMP_FLAG, model.GROUP_FLAG, model.PREV_GROUP_FLAG, model.FULL_SEASON_FLAG, model.PREV_FULL_SEASON_FLAG, model.PARTIAL_SEASON_FLAG, model.PREV_PARTIAL_SEASON_FLAG, model.prev_yr_events_attended, model.DISTANCE_TO_STADIUM_C FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model WHERE model.seasonyear LIKE ANY (2021, 2022, 2023)) as main LEFT JOIN (SELECT id_table.DW_ID, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = '6-Single Advance' THEN 1 ELSE 0 END) AS COUNT_SINGLE_ADVANCE, SUM(CASE WHEN arch.TICKET_TYPE_CATEGORY = 'Complimentary' THEN 1 ELSE 0 END) AS COUNT_COMPLIMENTARY FROM (SELECT model.DW_ID, master.ACCT_ID_T FROM CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model LEFT JOIN CHICAGOFIRE_DATABASE.MLS.MASTER_CF as master ON model.DW_ID = master.DW_ID_MASTER WHERE model.SEASONYEAR in (2021, 2022)) AS id_table INNER JOIN CHICAGOFIRE_DATABASE.ARCHTICS.ARCHITCS_2022 as arch ON id_table.ACCT_ID_T = arch.ACCT_ID GROUP BY id_table.DW_ID) as sub ON main.DW_ID = sub.DW_ID LEFT JOIN (SELECT model.DW_ID, SUM(CASE WHEN task.TASKSUBTYPE = 'Email' THEN 1 ELSE 0 END) AS TASK_COUNT_EMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'List Email' THEN 1 ELSE 0 END) AS TASK_COUNT_LISTEMAIL, SUM(CASE WHEN task.TASKSUBTYPE = 'Call' THEN 1 ELSE 0 END) AS TASK_COUNT_CALL, SUM(CASE WHEN task.TASKSUBTYPE = 'Task' THEN 1 ELSE 0 END) AS TASK_COUNT_TASK, SUM(CASE WHEN task.TASKSUBTYPE = 'Cadence' THEN 1 ELSE 0 END) AS TASK_COUNT_CADENCE FROM CHICAGOFIRE_DATABASE.SALESFORCE.SF_OPPORTUNITIES as opp INNER JOIN CHICAGOFIRE_DATABASE.SALESFORCE.SF_TASK as task ON task.WHOID = opp.CONTACTID__C INNER JOIN CHICAGOFIRE_DATABASE.MLS.MODEL_CF as model ON model.ACCT_ID_T = opp.TICKET_ID__C WHERE (task.ACTIVITYDATE BETWEEN '2022-08-01' AND '2023-06-01') and opp.ticket_id__c not like 'napanagopoulos@gmail.com'GROUP BY model.DW_ID) as taskdata ON main.DW_ID = taskdata.DW_ID"
res2 = conn.cursor().execute(sql2).fetchall()
df2 = pd.read_sql(sql2, conn)
df23_ids = df2[df2.SEASONYEAR == '2023']

C:\Users\dgilbert\AppData\Local\Temp\ipykernel_20388\3426509197.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(sql2, conn)


In [ ]:
df23_ids

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,PREV_PARTIAL_SEASON_FLAG,PREV_YR_EVENTS_ATTENDED,DISTANCE_TO_STADIUM_C,COUNT_SINGLE_ADVANCE,COUNT_COMPLIMENTARY,TASK_COUNT_EMAIL,TASK_COUNT_LISTEMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK,TASK_COUNT_CADENCE
1,5814267,Maria,Lysandrou,2023,0,3138,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5814273,Alberto,Ramos,2023,0,2358,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5814277,Katie,Thompson,2023,0,3755,0,0,0,0.0,...,0,0,392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,5814285,None,Blank,2023,0,2495,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,5814317,Peter,Kolb,2023,0,1433,0,0,0,0.0,...,0,0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948218,15288521,John,Strange,2023,0,576,0,0,0,0.0,...,0,1,54.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2948221,15288533,None,None,2023,0,557,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2948224,15288556,None,None,2023,0,557,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2948227,15288590,None,None,2023,0,557,0,0,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ids = df_ids['Dw Id Master (MASTER_CF)']
ids = ids.tolist()
ids2 = [339699, 17476891, 5433549, 13150752, 4105750, 860556, 11378652]

In [ ]:
df_ids_fill = df23_ids[df23_ids['DW_ID'].isin(ids2)]

In [ ]:
df_ids_fill

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,PREV_PARTIAL_SEASON_FLAG,PREV_YR_EVENTS_ATTENDED,DISTANCE_TO_STADIUM_C,COUNT_SINGLE_ADVANCE,COUNT_COMPLIMENTARY,TASK_COUNT_EMAIL,TASK_COUNT_LISTEMAIL,TASK_COUNT_CALL,TASK_COUNT_TASK,TASK_COUNT_CADENCE
150357,11378652,Wally,Lagerwey,2023,0,9353,0,0,2,0.00,...,0,12,17.0,0.0,0.0,240.0,0.0,120.0,0.0,0.0
719613,5433549,Lauren,Morris,2023,0,3705,0,4,8,0.00,...,0,6,11.0,0.0,0.0,90.0,0.0,270.0,30.0,0.0
1208842,17476891,Carlos,Contreras,2023,0,211,0,0,0,0.00,...,0,0,2.0,NaN,NaN,0.0,0.0,60.0,0.0,0.0
1663458,4105750,Jade,Lamoreaux,2023,0,2145,0,0,0,0.00,...,0,0,6.0,NaN,NaN,40.0,0.0,160.0,120.0,0.0
1815984,339699,Sebastian,Pollandt,2023,1,2571,1,1,7,170.11,...,0,4,4.0,0.0,0.0,80.0,0.0,30.0,0.0,0.0
1939056,13150752,Eric,Nelson,2023,0,821,0,0,0,0.00,...,0,3,4.0,0.0,0.0,150.0,0.0,240.0,60.0,0.0
2185308,860556,Dave,Williams,2023,0,6023,0,0,0,0.00,...,0,4,17.0,0.0,0.0,20.0,0.0,240.0,40.0,0.0


In [ ]:
df23_ids['MLS_TENURE_YEARS'] = df23_ids['MLS_TENURE']/365.242374
df23_ids['ATTENDANCE_RATE'] = np.nan
# Convert the SEASONYEAR column to integer
df23_ids['SEASONYEAR'] = df23_ids['SEASONYEAR'].astype('int64')
#df_features.loc[df_features.SEASONYEAR == 2021, 'ATTENDANCE_RATE'] = df_features.loc[df_features.SEASONYEAR == 2021, 'ANNUAL_EVENTS_ATTENDED']/games_21
df23_ids.loc[df23_ids.SEASONYEAR == 2023, 'ATTENDANCE_RATE'] = df23_ids.loc[df23_ids.SEASONYEAR == 2023, 'ATTENDED_EVENTS']/games_23 # should be 'ANNUAL_EVENTS_ATTENDED'. Issue w data rn
df23_ids[['SEASONYEAR', 'ATTENDED_EVENTS', 'ATTENDANCE_RATE']]#, 'RENEWED_NEXT_YEAR', 'MLS_TENURE_YEARS']]
df23_ids['AVG_ORDER_SPEND'] = df23_ids['ORDER_SPEND_M']/df23_ids['ORDER_QUANTITY_M']
df23_ids['AVG_ORDER_SPEND'] = df23_ids['AVG_ORDER_SPEND'].fillna(0)
df23_ids['AVG_ORDER_SPEND_CAT'] = df23_ids['AVG_ORDER_SPEND'].apply(categorize_spend)
df23_ids = pd.get_dummies(df23_ids, columns=['AVG_ORDER_SPEND_CAT'])

C:\Users\dgilbert\AppData\Local\Temp\ipykernel_20388\3829188403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23_ids['MLS_TENURE_YEARS'] = df23_ids['MLS_TENURE']/365.242374
C:\Users\dgilbert\AppData\Local\Temp\ipykernel_20388\3829188403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23_ids['ATTENDANCE_RATE'] = np.nan
C:\Users\dgilbert\AppData\Local\Temp\ipykernel_20388\3829188403.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [ ]:
df_ids_fill['MLS_TENURE_YEARS'] = df_ids_fill['MLS_TENURE']/365.242374
df_ids_fill['ATTENDANCE_RATE'] = np.nan
# Convert the SEASONYEAR column to integer
df_ids_fill['SEASONYEAR'] = df_ids_fill['SEASONYEAR'].astype('int64')
#df_features.loc[df_features.SEASONYEAR == 2021, 'ATTENDANCE_RATE'] = df_features.loc[df_features.SEASONYEAR == 2021, 'ANNUAL_EVENTS_ATTENDED']/games_21
df_ids_fill.loc[df_ids_fill.SEASONYEAR == 2023, 'ATTENDANCE_RATE'] = df_ids_fill.loc[df_ids_fill.SEASONYEAR == 2023, 'ATTENDED_EVENTS']/games_23 # should be 'ANNUAL_EVENTS_ATTENDED'. Issue w data rn
df_ids_fill[['SEASONYEAR', 'ATTENDED_EVENTS', 'ATTENDANCE_RATE']]#, 'RENEWED_NEXT_YEAR', 'MLS_TENURE_YEARS']]
df_ids_fill['AVG_ORDER_SPEND'] = df_ids_fill['ORDER_SPEND_M']/df_ids_fill['ORDER_QUANTITY_M']
df_ids_fill['AVG_ORDER_SPEND'] = df_ids_fill['AVG_ORDER_SPEND'].fillna(0)
df_ids_fill['AVG_ORDER_SPEND_CAT'] = df_ids_fill['AVG_ORDER_SPEND'].apply(categorize_spend)
df_ids_fill = pd.get_dummies(df_ids_fill, columns=['AVG_ORDER_SPEND_CAT'])

C:\Users\dgilbert\AppData\Local\Temp\ipykernel_34336\1976067618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ids_fill['MLS_TENURE_YEARS'] = df_ids_fill['MLS_TENURE']/365.242374
C:\Users\dgilbert\AppData\Local\Temp\ipykernel_34336\1976067618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ids_fill['ATTENDANCE_RATE'] = np.nan
C:\Users\dgilbert\AppData\Local\Temp\ipykernel_34336\1976067618.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [ ]:
df_ids_fill.columns

Index(['DW_ID', 'FIRST_NAME_C', 'LAST_NAME_C', 'SEASONYEAR',
       'RENEWED_NEXT_YEAR', 'MLS_TENURE', 'ORDER_QUANTITY_M',
       'ORDER_QUANTITY_M_LAST_YEAR', 'ORDER_QUANTITY_M_ALL_TIME',
       'ORDER_SPEND_M', 'ORDER_SPEND_M_LAST_YEAR', 'ORDER_SPEND_M_ALL_TIME',
       'TICKET_QUANTITY_T', 'TICKET_QUANTITY_LAST_YEAR',
       'TICKET_QUANTITY_ALL_TIME', 'ATTENDED_EVENTS',
       'ATTENDED_EVENTS_LAST_YEAR', 'ATTENDED_EVENTS_ALL_TIME',
       'TICKET_SPEND_T', 'TICKET_SPEND_T_LAST_YEAR', 'TICKET_SPEND_T_ALL_TIME',
       'SINGLE_GAME_FLAG', 'PREV_SINGLE_GAME_FLAG', 'COMP_FLAG',
       'PREV_COMP_FLAG', 'GROUP_FLAG', 'PREV_GROUP_FLAG', 'FULL_SEASON_FLAG',
       'PREV_FULL_SEASON_FLAG', 'PARTIAL_SEASON_FLAG',
       'PREV_PARTIAL_SEASON_FLAG', 'PREV_YR_EVENTS_ATTENDED',
       'DISTANCE_TO_STADIUM_C', 'COUNT_SINGLE_ADVANCE', 'COUNT_COMPLIMENTARY',
       'TASK_COUNT_EMAIL', 'TASK_COUNT_LISTEMAIL', 'TASK_COUNT_CALL',
       'TASK_COUNT_TASK', 'TASK_COUNT_CADENCE', 'MLS_TENURE_YEARS',
  

In [ ]:
df_ids_fill['COUNT_SINGLE_ADVANCE'] = df_ids_fill['COUNT_SINGLE_ADVANCE'].fillna(0).astype(int)
df_ids_fill['COUNT_COMPLIMENTARY'] = df_ids_fill['COUNT_COMPLIMENTARY'].fillna(0).astype(int)
df_ids_fill['TASK_COUNT_EMAIL'] = df_ids_fill['TASK_COUNT_EMAIL'].fillna(0).astype(int)
df_ids_fill['TASK_COUNT_LISTEMAIL'] = df_ids_fill['TASK_COUNT_LISTEMAIL'].fillna(0).astype(int)
df_ids_fill['TASK_COUNT_CALL'] = df_ids_fill['TASK_COUNT_CALL'].fillna(0).astype(int)
df_ids_fill['TASK_COUNT_TASK'] = df_ids_fill['TASK_COUNT_TASK'].fillna(0).astype(int)
df_ids_fill['TASK_COUNT_CADENCE'] = df_ids_fill['TASK_COUNT_CADENCE'].fillna(0).astype(int)

In [ ]:
df23_ids['COUNT_SINGLE_ADVANCE'] = df23_ids['COUNT_SINGLE_ADVANCE'].fillna(0).astype(int)
df23_ids['COUNT_COMPLIMENTARY'] = df23_ids['COUNT_COMPLIMENTARY'].fillna(0).astype(int)
df23_ids['TASK_COUNT_EMAIL'] = df23_ids['TASK_COUNT_EMAIL'].fillna(0).astype(int)
df23_ids['TASK_COUNT_LISTEMAIL'] = df23_ids['TASK_COUNT_LISTEMAIL'].fillna(0).astype(int)
df23_ids['TASK_COUNT_CALL'] = df23_ids['TASK_COUNT_CALL'].fillna(0).astype(int)
df23_ids['TASK_COUNT_TASK'] = df23_ids['TASK_COUNT_TASK'].fillna(0).astype(int)
df23_ids['TASK_COUNT_CADENCE'] = df23_ids['TASK_COUNT_CADENCE'].fillna(0).astype(int)

In [ ]:
#df_ids_fill['AVG_ORDER_SPEND_CAT_High'] = df_ids_fill['AVG_ORDER_SPEND_CAT_High'].astype(int)
df_ids_fill['AVG_ORDER_SPEND_CAT_Medium'] = df_ids_fill['AVG_ORDER_SPEND_CAT_Medium'].astype(int)
#df_ids_fill['AVG_ORDER_SPEND_CAT_Low'] = df_ids_fill['AVG_ORDER_SPEND_CAT_Low'].astype(int)
df_ids_fill['AVG_ORDER_SPEND_CAT_No Purchase'] = df_ids_fill['AVG_ORDER_SPEND_CAT_No Purchase'].astype(int)
df_ids_fill['TENURE_LOG'] = np.log(df_ids_fill['MLS_TENURE_YEARS']).replace(-np.inf, 0)
df_ids_fill['TICKET_SPEND_PER_GAME'] = df_ids_fill['TICKET_SPEND_T']/games_23
df_ids_fill['TICKET_SPEND_PER_GAME_LOG'] = np.log(df_ids_fill['TICKET_SPEND_PER_GAME']).replace(-np.inf, 0)
df_ids_fill['COUNT_SINGLE_ADVANCE_LOG'] = np.log(df_ids_fill['COUNT_SINGLE_ADVANCE']).replace(-np.inf, 0)
df_ids_fill['COUNT_COMPLIMENTARY_LOG'] = np.log(df_ids_fill['COUNT_COMPLIMENTARY']).replace(-np.inf, 0)
df_ids_fill['TASK_COUNT_EMAIL_LOG'] = np.log(df_ids_fill['TASK_COUNT_EMAIL']).replace(-np.inf, 0)
df_ids_fill['TASK_COUNT_CALL_LOG'] = np.log(df_ids_fill['TASK_COUNT_CALL']).replace(-np.inf, 0)
df_ids_fill['TASK_COUNT_TASK_LOG'] = np.log(df_ids_fill['TASK_COUNT_TASK']).replace(-np.inf, 0)
df_ids_fill['TASK_COUNT_CADENCE_LOG'] = np.log(df_ids_fill['TASK_COUNT_CADENCE']).replace(-np.inf, 0)

c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result

In [ ]:
df23_ids['AVG_ORDER_SPEND_CAT_High'] = df23_ids['AVG_ORDER_SPEND_CAT_High'].astype(int)
df23_ids['AVG_ORDER_SPEND_CAT_Medium'] = df23_ids['AVG_ORDER_SPEND_CAT_Medium'].astype(int)
df23_ids['AVG_ORDER_SPEND_CAT_Low'] = df23_ids['AVG_ORDER_SPEND_CAT_Low'].astype(int)
df23_ids['AVG_ORDER_SPEND_CAT_No Purchase'] = df23_ids['AVG_ORDER_SPEND_CAT_No Purchase'].astype(int)
df23_ids['TENURE_LOG'] = np.log(df23_ids['MLS_TENURE_YEARS']).replace(-np.inf, 0)
df23_ids['TICKET_SPEND_PER_GAME'] = df23_ids['TICKET_SPEND_T']/games_23
df23_ids['TICKET_SPEND_PER_GAME_LOG'] = np.log(df23_ids['TICKET_SPEND_PER_GAME']).replace(-np.inf, 0)
df23_ids['COUNT_SINGLE_ADVANCE_LOG'] = np.log(df23_ids['COUNT_SINGLE_ADVANCE']).replace(-np.inf, 0)
df23_ids['COUNT_COMPLIMENTARY_LOG'] = np.log(df23_ids['COUNT_COMPLIMENTARY']).replace(-np.inf, 0)
df23_ids['TASK_COUNT_EMAIL_LOG'] = np.log(df23_ids['TASK_COUNT_EMAIL']).replace(-np.inf, 0)
df23_ids['TASK_COUNT_CALL_LOG'] = np.log(df23_ids['TASK_COUNT_CALL']).replace(-np.inf, 0)
df23_ids['TASK_COUNT_TASK_LOG'] = np.log(df23_ids['TASK_COUNT_TASK']).replace(-np.inf, 0)
df23_ids['TASK_COUNT_CADENCE_LOG'] = np.log(df23_ids['TASK_COUNT_CADENCE']).replace(-np.inf, 0)

c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\dgilbert\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result

In [ ]:
df_ids_fill['AVG_ORDER_SPEND_CAT_High'] = 0
df_ids_fill['AVG_ORDER_SPEND_CAT_Low'] = 0

In [ ]:
df_ids_fill

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,TICKET_SPEND_PER_GAME,TICKET_SPEND_PER_GAME_LOG,COUNT_SINGLE_ADVANCE_LOG,COUNT_COMPLIMENTARY_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG,TASK_COUNT_CADENCE_LOG,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Low
150357,11378652,Wally,Lagerwey,2023,0,9353,0,0,2,0.00,...,53.518182,3.980021,0.0,0.0,5.480639,4.787492,0.000000,0.0,0,0
719613,5433549,Lauren,Morris,2023,0,3705,0,4,8,0.00,...,15.097273,2.714514,0.0,0.0,4.499810,5.598422,3.401197,0.0,0,0
1208842,17476891,Carlos,Contreras,2023,0,211,0,0,0,0.00,...,20.000000,2.995732,0.0,0.0,0.000000,4.094345,0.000000,0.0,0,0
1663458,4105750,Jade,Lamoreaux,2023,0,2145,0,0,0,0.00,...,7.556818,2.022450,0.0,0.0,3.688879,5.075174,4.787492,0.0,0,0
1815984,339699,Sebastian,Pollandt,2023,1,2571,1,1,7,170.11,...,36.590909,3.599800,0.0,0.0,4.382027,3.401197,0.000000,0.0,0,0
1939056,13150752,Eric,Nelson,2023,0,821,0,0,0,0.00,...,12.272727,2.507380,0.0,0.0,5.010635,5.480639,4.094345,0.0,0,0
2185308,860556,Dave,Williams,2023,0,6023,0,0,0,0.00,...,0.000000,0.000000,0.0,0.0,2.995732,5.480639,3.688879,0.0,0,0


In [ ]:
X_predict = df_ids_fill[['ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME_LOG', 
                 'TENURE_LOG', #
                 'COUNT_SINGLE_ADVANCE_LOG', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL_LOG', 
                'TASK_COUNT_CALL_LOG', 'TASK_COUNT_TASK_LOG'
                #, 'TASK_COUNT_CADENCE_LOG'
                 ]]

predictions_2023_remains = model.predict(X_predict)

In [ ]:
df_ids_fill[['ATTENDANCE_RATE', 'AVG_ORDER_SPEND_CAT_High', 'AVG_ORDER_SPEND_CAT_Medium', 'AVG_ORDER_SPEND_CAT_Low', 'AVG_ORDER_SPEND_CAT_No Purchase', 
                 'TICKET_SPEND_PER_GAME_LOG', 
                 'TENURE_LOG', 
                 'COUNT_SINGLE_ADVANCE_LOG', #'COUNT_COMPLIMENTARY_LOG', 
                 'TASK_COUNT_EMAIL_LOG', 
                 'TASK_COUNT_CALL_LOG', 'TASK_COUNT_TASK_LOG'
                #, 'TASK_COUNT_CADENCE_LOG'
                 ]]

,ATTENDANCE_RATE,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Medium,AVG_ORDER_SPEND_CAT_Low,AVG_ORDER_SPEND_CAT_No Purchase,TICKET_SPEND_PER_GAME_LOG,TENURE_LOG,COUNT_SINGLE_ADVANCE_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG
13329,0.136364,0,0,0,1,3.649382,2.651033,0.0,4.605170,6.253829,3.688879
25242,0.000000,0,0,0,1,2.715537,1.342952,0.0,4.382027,6.396930,4.605170
52779,0.000000,0,0,0,1,2.897819,2.116417,0.0,3.401197,4.787492,4.094345
144342,0.090909,0,0,0,1,2.455111,1.302100,0.0,5.192957,6.234411,5.010635
155343,0.181818,0,1,0,0,3.555348,1.352201,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2814224,0.181818,0,0,0,1,2.091303,-1.017759,0.0,4.094345,5.886104,4.499810
2815205,0.090909,0,0,0,1,2.219697,-1.040749,0.0,2.995732,3.688879,0.000000
2815370,0.045455,0,0,0,1,2.378873,-1.040749,0.0,4.094345,4.382027,2.995732
2817578,0.090909,0,0,0,1,2.849129,-0.399303,0.0,4.499810,3.401197,0.000000


In [ ]:
df_ids_fill['PREDICTIONS'] = predictions_2023_remains
probabilities_2023_remains = model.predict_proba(X_predict)
df_ids_fill['renewal_predict_prob'] = probabilities_2023_remains[:, 1]

In [ ]:
df_ids_fill

,DW_ID,FIRST_NAME_C,LAST_NAME_C,SEASONYEAR,RENEWED_NEXT_YEAR,MLS_TENURE,ORDER_QUANTITY_M,ORDER_QUANTITY_M_LAST_YEAR,ORDER_QUANTITY_M_ALL_TIME,ORDER_SPEND_M,...,COUNT_SINGLE_ADVANCE_LOG,COUNT_COMPLIMENTARY_LOG,TASK_COUNT_EMAIL_LOG,TASK_COUNT_CALL_LOG,TASK_COUNT_TASK_LOG,TASK_COUNT_CADENCE_LOG,AVG_ORDER_SPEND_CAT_High,AVG_ORDER_SPEND_CAT_Low,PREDICTIONS,renewal_predict_prob
150357,11378652,Wally,Lagerwey,2023,0,9353,0,0,2,0.00,...,0.0,0.0,5.480639,4.787492,0.000000,0.0,0,0,1,0.890189
719613,5433549,Lauren,Morris,2023,0,3705,0,4,8,0.00,...,0.0,0.0,4.499810,5.598422,3.401197,0.0,0,0,0,0.488697
1208842,17476891,Carlos,Contreras,2023,0,211,0,0,0,0.00,...,0.0,0.0,0.000000,4.094345,0.000000,0.0,0,0,0,0.175519
1663458,4105750,Jade,Lamoreaux,2023,0,2145,0,0,0,0.00,...,0.0,0.0,3.688879,5.075174,4.787492,0.0,0,0,0,0.206160
1815984,339699,Sebastian,Pollandt,2023,1,2571,1,1,7,170.11,...,0.0,0.0,4.382027,3.401197,0.000000,0.0,0,0,1,0.814927
1939056,13150752,Eric,Nelson,2023,0,821,0,0,0,0.00,...,0.0,0.0,5.010635,5.480639,4.094345,0.0,0,0,0,0.262263
2185308,860556,Dave,Williams,2023,0,6023,0,0,0,0.00,...,0.0,0.0,2.995732,5.480639,3.688879,0.0,0,0,0,0.149295


In [ ]:
df_ids_fill[['DW_ID', 'FIRST_NAME_C', 'LAST_NAME_C', 'renewal_predict_prob']].to_csv("Output_Finale_pls.csv")

In [ ]:
df_output = pd.read_csv("2024 Renewals_'24 Universe Master_20230811T1415 with Predictions Phase 3.csv")
ids = df_output["DW_ID (Predictions Phase 3.csv)"][10:]

In [ ]:
mask = df23_ids['DW_ID'].isin(ids)

In [ ]:
# Using the mask to filter df2
filtered_df23 = df23_ids[mask]



In [ ]:
filtered_df23.to_csv("output_output.csv")